In [1]:
# STEP 0: Setup & Spark
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), "../../")))

from config import settings
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, trim

spark = SparkSession.builder \
    .appName("Synthetic_ETL_Pipeline") \
    .config("spark.jars", settings.JDBC_PATH) \
    .getOrCreate()

print("✅ Spark session started.")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/19 19:58:59 WARN Utils: Your hostname, Nagendras-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.17 instead (on interface en0)
25/06/19 19:58:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/06/19 19:58:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
/Users/aryan/Desktop/project/venv/lib/python3.9/site-packages/google/rpc/__init__.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin t

✅ Spark session started.


In [7]:
import os
import sys
from pyspark.sql import SparkSession

# Set manually for notebooks (since __file__ doesn't exist)
BASE_DIR = os.path.abspath("../../")

sys.path.append(BASE_DIR)
from config.spark_config import get_spark_session

spark = get_spark_session("Read_Parquet_Customers")

parquet_path = os.path.join(BASE_DIR, "data", "parquet_data", "customers_transformed.parquet")

if not os.path.exists(parquet_path):
    raise FileNotFoundError(f"❌ Parquet file not found at: {parquet_path}")
else:
    print(f"✅ Reading Parquet file from: {parquet_path}")

df = spark.read.parquet(parquet_path)

df.printSchema()
df.show(truncate=False)



✅ Reading Parquet file from: /Users/aryan/Desktop/project/data/parquet_data/customers_transformed.parquet
root
 |-- customer_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- company: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- phone1: string (nullable = true)
 |-- phone2: string (nullable = true)
 |-- email: string (nullable = true)
 |-- subscription_date: date (nullable = true)
 |-- website: string (nullable = true)
 |-- full_name: string (nullable = true)
 |-- email_lower: string (nullable = true)
 |-- country_upper: string (nullable = true)
 |-- name_length: integer (nullable = true)
 |-- ingested_at: timestamp (nullable = true)

+---------------+----------+-----------+----------------------------+----------------+---------------------------------------------------+---------------------+----------------------+-----------------------------+-----------------

In [8]:
df.count()

2000000

In [11]:
from pyspark.sql.functions import upper

# Filter rows where country_upper == 'INDIA'
df_india = df.filter(upper(df["country_upper"]) == "INDIA")

# Show result
df_india.show(truncate=False)
df_india.count()

+---------------+----------+---------+------------------------------+-------------------+-------+----------------------+----------------------+-----------------------------------+-----------------+-------------------------------+-----------------+-----------------------------------+-------------+-----------+--------------------------+
|customer_id    |first_name|last_name|company                       |city               |country|phone1                |phone2                |email                              |subscription_date|website                        |full_name        |email_lower                        |country_upper|name_length|ingested_at               |
+---------------+----------+---------+------------------------------+-------------------+-------+----------------------+----------------------+-----------------------------------+-----------------+-------------------------------+-----------------+-----------------------------------+-------------+-----------+-----------------

8279

25/06/19 20:41:51 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 935421 ms exceeds timeout 120000 ms
25/06/19 20:41:51 WARN SparkContext: Killing executors is not supported by current scheduler.
25/06/19 20:41:56 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:81)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:669)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1296)
	at o